In [1]:
import pandas as pd
import sklearn.metrics as metrique
from pandas import Series
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dense, Embedding, Dropout,Input, Attention, Layer, Concatenate, Permute, Dot, Multiply, Flatten
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
from tensorflow.keras import backend as K, regularizers, Model, metrics
from tensorflow.keras.backend import cast
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
X_train = pd.read_csv('324 X_resampled  + randomly distributed Gan + EllipticEnvelope+ duplicate remover.csv', na_filter=True)
y_train = pd.read_csv('324 y_resampled  + randomly distributed Gan + EllipticEnvelope+ duplicate remover.csv', na_filter=True)
X_test = pd.read_csv('324 test  X_resampled  + Gan + EllipticEnvelope.csv', na_filter=True)
y_test = pd.read_csv('324 test  y_resampled  + Gan + EllipticEnvelope.csv', na_filter=True)

In [4]:
n=2

In [5]:
X_train0,X_train1=np.array_split(X_train,n)
y_train0,y_train1=np.array_split(y_train,n)

In [13]:
y_train0.value_counts()

0.0    88719
1.0      165
Name: Class, dtype: int64

In [7]:
X_train0.shape,X_train1.shape

((98761, 18), (98760, 18))

In [8]:
y_train0.shape,y_train1.shape

((98761,), (98760,))

In [9]:
X_train0 = StandardScaler().fit_transform(X_train0)
X_train1 = StandardScaler().fit_transform(X_train1)
X_test=StandardScaler().fit_transform(X_test)

In [10]:
X_train0,X_val0,y_train0,y_val0 = train_test_split(X_train0, y_train0, test_size=0.1)
X_train1,X_val1,y_train1,y_val1 = train_test_split(X_train1, y_train1, test_size=0.1)

In [11]:
def making_data (X_train,X_val,X_test,y_train,y_val,y_test):
    # X_train et X_val sont des dataframe qui contient les features
    train_GRU_X=X_train
    val_GRU_X=X_val
    test_GRU_X=X_test

    ## Reshape input to be 3D [samples, timesteps, features] (format requis par GRU)
    train_GRU_X = train_GRU_X.reshape((train_GRU_X.shape[0], 1, train_GRU_X.shape[1]))
    val_GRU_X = val_GRU_X.reshape((val_GRU_X.shape[0], 1, val_GRU_X.shape[1]))
    test_GRU_X = test_GRU_X.reshape((test_GRU_X.shape[0], 1, test_GRU_X.shape[1]))

    ## Recuperation des labels
    train_GRU_y=y_train
    val_GRU_y=y_val
    test_GRU_y=y_test
    return train_GRU_X,val_GRU_X,test_GRU_X,train_GRU_y,val_GRU_y,test_GRU_y

# n=1

In [12]:
train_GRU_X0,val_GRU_X0,test_GRU_X,train_GRU_y0,val_GRU_y0,test_GRU_y=making_data (X_train0,X_val0,X_test,y_train0,y_val0,y_test)
model0 = Sequential()
model0.add(GRU(50,dropout=0.3,recurrent_dropout=0.2, return_sequences=True))
model0.add(Dense(1, activation='sigmoid'))
model0.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model0.fit(train_GRU_X0, train_GRU_y0,epochs=200,batch_size=2000, validation_data=(val_GRU_X0, val_GRU_y0))

Epoch 1/200
45/45 [==============================] - 3s 19ms/step - loss: 0.5824 - accuracy: 0.8361 - val_loss: 0.4440 - val_accuracy: 0.9903
Epoch 2/200
45/45 [==============================] - 1s 11ms/step - loss: 0.3456 - accuracy: 0.9904 - val_loss: 0.2152 - val_accuracy: 0.9989
Epoch 3/200
45/45 [==============================] - 1s 11ms/step - loss: 0.1688 - accuracy: 0.9979 - val_loss: 0.0960 - val_accuracy: 0.9986
Epoch 4/200
45/45 [==============================] - 1s 12ms/step - loss: 0.0844 - accuracy: 0.9984 - val_loss: 0.0489 - val_accuracy: 0.9987
Epoch 5/200
45/45 [==============================] - 1s 12ms/step - loss: 0.0481 - accuracy: 0.9988 - val_loss: 0.0294 - val_accuracy: 0.9990
Epoch 6/200
45/45 [==============================] - 1s 12ms/step - loss: 0.0313 - accuracy: 0.9990 - val_loss: 0.0201 - val_accuracy: 0.9990
Epoch 7/200
45/45 [==============================] - 1s 11ms/step - loss: 0.0221 - accuracy: 0.9990 - val_loss: 0.0151 - val_accuracy: 0.9990
Epoch 

KeyboardInterrupt: 

In [ ]:
# predict probabilities for test set
yhat_probs0 = model0.predict(test_GRU_X, verbose=0)> 0.5
# reduce to 1d array
yhat_probs0 = yhat_probs0[:, 0]

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_GRU_y, yhat_probs0)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_GRU_y, yhat_probs0)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_GRU_y, yhat_probs0)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_GRU_y, yhat_probs0)
print('f1: %f' % f1)

print(classification_report(test_GRU_y, yhat_probs0))

# n=2

In [ ]:
train_GRU_X1,val_GRU_X1,test_GRU_X,train_GRU_y1,val_GRU_y1,test_GRU_y=making_data (X_train1,X_val1,X_test,y_train1,y_val1,y_test)
model1 = Sequential()
model1.add(GRU(50,dropout=0.3,recurrent_dropout=0.2, return_sequences=True))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model1.fit(train_GRU_X1, train_GRU_y1,epochs=200,batch_size=2000, validation_data=(val_GRU_X1, val_GRU_y1))

In [ ]:
# predict probabilities for test set
yhat_probs1 = model1.predict(test_GRU_X, verbose=0)> 0.5
# reduce to 1d array
yhat_probs1 = yhat_probs1[:, 0]

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_GRU_y, yhat_probs1)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_GRU_y, yhat_probs1)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_GRU_y, yhat_probs1)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_GRU_y, yhat_probs1)
print('f1: %f' % f1)

print(classification_report(test_GRU_y, yhat_probs1))

# attention

In [ ]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
inputs1=Input((1,18))
att_in=GRU(50,return_sequences=True,dropout=0.3,recurrent_dropout=0.2)(inputs1)
#att_in_1=GRU(50,return_sequences=True,dropout=0.3,recurrent_dropout=0.2)(att_in)
att_out=attention()(att_in)
outputs1=Dense(1,activation='sigmoid',trainable=True)(att_out)

# n=1

In [ ]:
model10=Model(inputs1,outputs1)
model10.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history10=model10.fit(train_GRU_X0, train_GRU_y0,epochs=200,batch_size=3000, validation_data=(val_GRU_X0, val_GRU_y0))

In [ ]:
# predict probabilities for test set
yhat_probs10 = model10.predict(test_GRU_X, verbose=0)> 0.5
# reduce to 1d array
yhat_probs10 = yhat_probs10[:, 0]

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_GRU_y, yhat_probs10)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_GRU_y, yhat_probs10)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_GRU_y, yhat_probs10)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_GRU_y, yhat_probs10)
print('f1: %f' % f1)

print(classification_report(test_GRU_y, yhat_probs10))


# n=2

In [ ]:
model11=Model(inputs1,outputs1)
model11.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history11=model10.fit(train_GRU_X1, train_GRU_y1,epochs=200,batch_size=3000, validation_data=(val_GRU_X1, val_GRU_y1))

In [ ]:
# predict probabilities for test set
yhat_probs11 = model11.predict(test_GRU_X, verbose=0)> 0.5
# reduce to 1d array
yhat_probs11 = yhat_probs11[:, 0]

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_GRU_y, yhat_probs11)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_GRU_y, yhat_probs11)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_GRU_y, yhat_probs11)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_GRU_y, yhat_probs11)
print('f1: %f' % f1)

print(classification_report(test_GRU_y, yhat_probs11))


# FFNN

In [ ]:
X_train = StandardScaler().fit_transform(X_train)
X_train=X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))

## simple

In [ ]:
yhat_probs0 = model0.predict(X_train, verbose=0)
yhat_probs1 = model1.predict(X_train, verbose=0)
yhat_probs0=yhat_probs0.reshape(-1,1)
yhat_probs1=yhat_probs1.reshape(-1,1)

In [ ]:
X=np.concatenate((yhat_probs0,yhat_probs1),axis=1)

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier().fit(X, y_train)

In [ ]:
yhat_probs0 = model0.predict(test_GRU_X, verbose=0)
yhat_probs1 = model1.predict(test_GRU_X, verbose=0)
yhat_probs0=yhat_probs0.reshape(-1,1)
yhat_probs1=yhat_probs1.reshape(-1,1)
X_test=np.concatenate((yhat_probs0,yhat_probs1),axis=1)
y_pred=clf.predict(X_test)

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_GRU_y, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_GRU_y, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_GRU_y, y_pred)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_GRU_y, y_pred)
print('f1: %f' % f1)

print(classification_report(test_GRU_y, y_pred))


## attension

In [ ]:
yhat_probs10 = model10.predict(X_train, verbose=0)
yhat_probs11 = model11.predict(X_train, verbose=0)

In [ ]:
X=np.concatenate((yhat_probs10,yhat_probs11),axis=1)
X.shape

In [ ]:
clf = MLPClassifier().fit(X, y_train)

In [ ]:
yhat_probs10 = model10.predict(test_GRU_X, verbose=0)
yhat_probs11 = model11.predict(test_GRU_X, verbose=0)
X_test=np.concatenate((yhat_probs10,yhat_probs11),axis=1)
y_pred=clf.predict(X_test)

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_GRU_y, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_GRU_y, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_GRU_y, y_pred)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_GRU_y, y_pred)
print('f1: %f' % f1)
print(classification_report(test_GRU_y, y_pred))